# Week6 Exercise (group): Exploratory Data Analysis on Social Media Data

- Lori Lou
- Nayeon Bae
- Fai slavianto
- Ruiya Sun

## 1. Import necessary packages

In [72]:
!pip install emoji
!pip install nltk
!pip install textblob

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import emoji
from nltk.corpus import stopwords
from string import punctuation
from textblob import TextBlob

## 2. Read the data

The data is called `tweets.csv` in the same folder. More information about the data see [here](https://www.kaggle.com/datasets/infamouscoder/mental-health-social-media)

The main column you will be working with is `post_text`

In [73]:
df = pd.read_csv("tweets.csv")
df

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,819336825231773698,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1169875706,442,230,7,1063601,0,0
19996,19996,819334654260080640,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1169875706,442,230,7,1063601,0,0
19997,19997,819334503042871297,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1169875706,442,230,7,1063601,0,0
19998,19998,819334419374899200,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1169875706,442,230,7,1063601,0,0


In [74]:
#print the first 31 row and perform an emoji count
print(df['post_text'].head(31))
df['emoji_count'] = df['post_text'].apply(emoji.emoji_count)
print(df['emoji_count'].head(31))

0     It's just over 2 years since I was diagnosed w...
1     It's Sunday, I need a break, so I'm planning t...
2     Awake but tired. I need to sleep but my brain ...
3     RT @SewHQ: #Retro bears make perfect gifts and...
4     It’s hard to say whether packing lists are mak...
5     Making packing lists is my new hobby... #movin...
6     At what point does keeping stuff for nostalgic...
7     Currently in the finding-boxes-of-random-shit ...
8     Can't be bothered to cook, take away on the wa...
9     RT @itventsnews: ITV releases promo video for ...
10    ... also, I have too much stuff. Way, way too ...
11    I never want to put one of these together agai...
12    Moving stuff is bloomin’ knackering... and the...
13    Back at the house, moving stuff. It’s so peace...
14    Urgh. Anxiety. FFS where does it come from?! (...
15    I have too much stuff. Way, way too much... Ma...
16    Hideous traffic on the A14. Must remember to p...
17                  Packing and purging. Feels g

## 3. Extract emojis

Use `emoji` package to extract emojis and put them into a new column called `emojis`

In [75]:
def extract_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)

df['new_emojis'] = df['post_text'].apply(extract_emojis)
print(df['new_emojis'].head(31))

0         []
1         []
2         []
3         []
4         []
5         []
6         []
7         []
8      [😁👍🏼]
9         []
10        []
11        []
12       [😥]
13    [🏡, 😊]
14        []
15        []
16        []
17     [😊👍🏼]
18       [😉]
19        []
20        []
21        []
22       [💕]
23        []
24        []
25        []
26        []
27        []
28        []
29     [🏡🎉💕]
30        []
Name: new_emojis, dtype: object


## 4. Text Cleaning using Regular Expressions 

1. Remove URLs
2. Remove mentions
3. Remove hashtags
4. Remove special characters
5. Remove extra space

Code can be found in [week 6 lecture 1](https://github.com/yibeichan/COMM160DS/blob/main/week_6/lecture_part1.ipynb) section `4.4 All-in-One`

Perform the analysis and save the results into a new column.

In [76]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # remove URLs
    text = re.sub(r'@\w+', '', text)  # remove mentions
    text = re.sub(r'#\w+', '', text)  # remove hashtags
    text = re.sub(r'\W', ' ', text)  # remove special characters
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    return text.strip()

df = pd.read_csv("tweets.csv")

# Choose any text from the 'post_text' column
text = df['post_text'].iloc[17]  # Replace 0 with the index of the desired text

# Clean the chosen text
cleaned_text = clean_text(text)

print(cleaned_text)

Packing and purging Feels good


## 5. Analysis 1 (Rename the title with your chosen analysis)

Choose one analysis from (1)Sentiment Analysis 

Perform the analysis and save the results into a new column.

In [80]:
nltk.download('stopwords')

df = pd.read_csv("tweets.csv")
sentiments = []

# Iterate through all 31 locations in the 'post_text' column
for i in range(31):
    text = df['post_text'].iloc[i]  # Choose the text from each location
    
    # Clean the chosen text
    cleaned_text = clean_text(text)
    
    # Perform sentiment analysis
    sentiment = get_sentiment(cleaned_text)
    
    sentiments.append(sentiment)  # Append the sentiment to the list

# Print the sentiments for all 31 texts
for i, sentiment in enumerate(sentiments):
    print(f"Text {i+1} sentiment: {sentiment}")

Text 1 sentiment: positive
Text 2 sentiment: negative
Text 3 sentiment: negative
Text 4 sentiment: positive
Text 5 sentiment: negative
Text 6 sentiment: positive
Text 7 sentiment: negative
Text 8 sentiment: negative
Text 9 sentiment: neutral
Text 10 sentiment: neutral
Text 11 sentiment: positive
Text 12 sentiment: neutral
Text 13 sentiment: neutral
Text 14 sentiment: positive
Text 15 sentiment: neutral
Text 16 sentiment: positive
Text 17 sentiment: neutral
Text 18 sentiment: positive
Text 19 sentiment: positive
Text 20 sentiment: positive
Text 21 sentiment: negative
Text 22 sentiment: positive
Text 23 sentiment: positive
Text 24 sentiment: positive
Text 25 sentiment: positive
Text 26 sentiment: positive
Text 27 sentiment: positive
Text 28 sentiment: neutral
Text 29 sentiment: positive
Text 30 sentiment: neutral
Text 31 sentiment: negative


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 6. Analysis 2 (Rename the title with your chosen analysis)

(4)Part-of-Speech Tagging

Perform the analysis and save the results into a new column.

In [86]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

def pos_tagging(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)
    return tagged_tokens

df = pd.read_csv("tweets.csv")

# Iterate through all 31 locations in the 'post_text' column
for i in range(5):
    text = df['post_text'].iloc[i]  # Choose the text from each location
    
    # Clean the chosen text
    cleaned_text = clean_text(text)
    
    # Perform POS tagging
    pos_tags = pos_tagging(cleaned_text)
    
    print(f"Text {i+1} POS tags breakdown:")
    for token, pos in pos_tags:
        print(f"('{token}', '{pos}'): {token} is a {nltk.pos_tag([token])[0][1]}.")
    
    print()

Text 1 POS tags breakdown:
('It', 'PRP'): It is a PRP.
('s', 'VBZ'): s is a NN.
('just', 'RB'): just is a RB.
('over', 'IN'): over is a IN.
('2', 'CD'): 2 is a CD.
('years', 'NNS'): years is a NNS.
('since', 'IN'): since is a IN.
('I', 'PRP'): I is a PRP.
('was', 'VBD'): was is a VBD.
('diagnosed', 'VBN'): diagnosed is a VBN.
('with', 'IN'): with is a IN.
('and', 'CC'): and is a CC.
('Today', 'NNP'): Today is a NN.
('I', 'PRP'): I is a PRP.
('m', 'VBP'): m is a NN.
('taking', 'VBG'): taking is a VBG.
('a', 'DT'): a is a DT.
('moment', 'NN'): moment is a NN.
('to', 'TO'): to is a TO.
('reflect', 'VB'): reflect is a NN.
('on', 'IN'): on is a IN.
('how', 'WRB'): how is a WRB.
('far', 'RB'): far is a RB.
('I', 'PRP'): I is a PRP.
('ve', 'VBP'): ve is a NN.
('come', 'VBN'): come is a VB.
('since', 'IN'): since is a IN.

Text 2 POS tags breakdown:
('It', 'PRP'): It is a PRP.
('s', 'VBD'): s is a NN.
('Sunday', 'NNP'): Sunday is a NNP.
('I', 'PRP'): I is a PRP.
('need', 'VBP'): need is a NN.


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## 7. Push Your Results to GitHub

As you did in previous weeks:
1. `git status`
2. `git add`
3. `git commit -m "type your message here"`
4. `git push`

If you can't push it to GitHub, it's okay to manually uploaded it.